In [1]:
!pip install pdfplumber
!pip install langchain langchain-community
!pip install streamlit
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 831.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import streamlit as st

In [4]:
from google.colab import files

uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')


Saving goog-10-k-2023 (1).pdf to goog-10-k-2023 (1).pdf
Saving tsla-20231231-gen.pdf to tsla-20231231-gen.pdf
Saving uber-10-k-2023.pdf to uber-10-k-2023.pdf
User uploaded file "goog-10-k-2023 (1).pdf" with length 926655 bytes
User uploaded file "tsla-20231231-gen.pdf" with length 984581 bytes
User uploaded file "uber-10-k-2023.pdf" with length 1835051 bytes


In [12]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

# Paths to your PDF files
pdf_files = ["goog-10-k-2023 (1).pdf", "tsla-20231231-gen.pdf", "uber-10-k-2023.pdf"]

# Extract text from the uploaded PDFs
documents = [extract_text_from_pdf(file) for file in pdf_files]

# Check the length of the documents
for i, doc in enumerate(documents):
    print(f"Document {i + 1} length: {len(doc)}")


Document 1 length: 340173
Document 2 length: 436281
Document 3 length: 691178


In [13]:
# Print the first 500 characters of each document to verify the content
for i, doc in enumerate(documents):
    print(f"Document {i + 1} content preview: {doc[:500]}")


Document 1 content preview: UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
___________________________________________
FORM 10-K
___________________________________________
(Mark One)
☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2023
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from to .
Commission file number: 001-37580
________________________
Document 2 content preview: UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
x ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2023
OR
o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _________ to _________
Commission File Number: 001-34756
Tesla, Inc.
(Exact name of registrant

In [14]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Example model

# Generate embeddings for each document
embeddings = [model.encode(doc) for doc in documents]

# Verify embeddings shapes
for i, emb in enumerate(embeddings):
    print(f"Embedding {i + 1} shape: {emb.shape}")


Embedding 1 shape: (384,)
Embedding 2 shape: (384,)
Embedding 3 shape: (384,)


In [15]:
import faiss
import numpy as np

# Convert embeddings to a single 2D numpy array
embedding_matrix = np.array(embeddings, dtype=np.float32)

# Determine the dimension of the embeddings
dimension = embedding_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embedding_matrix)

# Save the index
faiss.write_index(index, "faiss_index.index")

print("FAISS index created and saved successfully.")


FAISS index created and saved successfully.


In [23]:
index = faiss.read_index("faiss_index.index")


In [24]:
documents = [Document(page_content=doc) for doc in ["Alphabet Inc.", "Tesla Inc.", "Uber Technologies Inc."]]


In [25]:
index_to_docstore_id = {i: i for i in range(len(documents))}


In [26]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [37]:
from langchain.docstore.in_memory import InMemoryDocstore

# Create an InMemoryDocstore with the documents
docstore = InMemoryDocstore(documents)

# Initialize LangChain's FAISS vector store
vector_store = LangChainFAISS(embedding_function=embeddings, docstore=docstore, index=index, index_to_docstore_id=index_to_docstore_id)


In [43]:
from langchain.vectorstores import FAISS as LangChainFAISS
from langchain.docstore.document import Document
import faiss
import numpy as np

# Load the FAISS index
index = faiss.read_index("faiss_index.index")

# Create documents with dummy texts for docstore
documents = [Document(page_content=doc) for doc in ["Alphabet Inc.", "Tesla Inc.", "Uber Technologies Inc."]]

# Define a custom docstore class implementing necessary methods
class CustomDocstore:
    def __init__(self, documents):
        self.documents = documents

    def search(self, doc_id):
        if doc_id < len(self.documents):
            return self.documents[doc_id]
        else:
            raise ValueError(f"Document with id {doc_id} not found.")

    def size(self):
        return len(self.documents)

# Define an example embedding function
def embedding_function(doc):
    # Replace with your actual logic to generate embeddings for the document
    return np.random.rand(384)  # Example placeholder, replace with actual logic

# Initialize the custom docstore
docstore = CustomDocstore(documents)

# Create a mapping of index to documents
index_to_docstore_id = {i: i for i in range(len(documents))}

# Initialize LangChain's FAISS vector store with embedding_function
vector_store = LangChainFAISS(docstore=docstore, index=index, index_to_docstore_id=index_to_docstore_id, embedding_function=embedding_function)

# Example query using LangChainFAISS
def query_engine(query):
    results = vector_store.similarity_search(query, k=5)
    return results

# Example queries and insights
query = "What are the risk factors associated with Google and Tesla?"
response = query_engine(query)
print("Query Result:", response)


Query Result: [Document(page_content='Tesla Inc.'), Document(page_content='Alphabet Inc.'), Document(page_content='Uber Technologies Inc.')]


In [44]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Initialize the LLM model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
llm_model = GPT2LMHeadModel.from_pretrained("gpt2")

def query_engine(query):
    results = vector_store.similarity_search(query, k=5)  # Adjust k based on your requirement
    return results

def generate_insights(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = llm_model.generate(inputs, max_length=200, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

# Example queries and insights
query = "What are the risk factors associated with Google and Tesla?"
response = query_engine(query)
print("Query Result:", response)

prompt = "Compare the business models of Tesla and Uber."
insight = generate_insights(prompt)
print("Insight:", insight)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query Result: [Document(page_content='Tesla Inc.'), Document(page_content='Alphabet Inc.'), Document(page_content='Uber Technologies Inc.')]
Insight: Compare the business models of Tesla and Uber.

The company has been in the news recently for its controversial decision to stop charging customers for rides on its Model S. The company has been criticized for charging customers for rides on its Model S, but the company has been able to get away with it because of its low cost.

Tesla has been criticized for charging customers for rides on its Model S, but the company has been able to get away with it because of its low cost.

Tesla has been criticized for charging customers for rides on its Model S, but the company has been able to get away with it because of its low cost.

Tesla has been criticized for charging customers for rides on its Model S, but the company has been able to get away with it because of its low cost.

Tesla has been criticized for charging customers for rides on its 

In [45]:
import streamlit as st

st.title("Content Engine for 10-K Filings")

query = st.text_input("Enter your query:")
if query:
    response = query_engine(query)
    st.write("Query Result:", response)

    prompt = st.text_input("Enter a prompt for detailed insight:")
    if prompt:
        insight = generate_insights(prompt)
        st.write("Insight:", insight)


2024-06-27 21:32:04.412 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-27 21:32:04.415 Session state does not function when running a script without `streamlit run`


In [48]:
import streamlit as st

# Function to query the content engine based on user input
def query_engine(query):
    # Replace with your logic to query the content engine
    # Example placeholder:
    results = f"Query results for '{query}'"
    return results

# Function to generate insights based on user prompt
def generate_insights(prompt):
    # Replace with your logic to generate insights
    # Example placeholder:
    insight = f"Insight for prompt: '{prompt}'"
    return insight

# Streamlit app title
st.title("Content Engine for 10-K Filings")

# User input for query
query = st.text_input("Enter your query:")
if query:
    response = query_engine(query)
    st.write("Query Result:", response)

    # User input for detailed insights
    prompt = st.text_input("Enter a prompt for detailed insight:")
    if prompt:
        insight = generate_insights(prompt)
        st.write("Insight:", insight)


In [51]:
!pip install streamlit




In [52]:
import streamlit as st

st.title("Content Engine for 10-K Filings")

query = st.text_input("Enter your query:")
if query:
    response = query_engine(query)
    st.write("Query Result:", response)

    prompt = st.text_input("Enter a prompt for detailed insight:")
    if prompt:
        insight = generate_insights(prompt)
        st.write("Insight:", insight)


In [53]:
import streamlit as st

st.title("Content Engine for 10-K Filings")

query = st.text_input("Enter your query:")
if query:
    response = query_engine(query)
    st.write("Query Result:", response)

    prompt = st.text_input("Enter a prompt for detailed insight:")
    if prompt:
        insight = generate_insights(prompt)
        st.write("Insight:", insight)


In [54]:
from google.colab import files
uploaded = files.upload()


Saving app.py to app.py


In [ ]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.27.60:8501

